In [1]:
import numpy as np
import pandas as pd
import nest
from scripts import optimization, neuron_categorization, visualizations


              -- N E S T --
  Copyright (C) 2004 The NEST Initiative

 Version: 3.6.0
 Built: Sep 28 2023 11:53:34

 This program is provided AS IS and comes with
 NO WARRANTY. See the file LICENSE for details.

 Problems or suggestions?
   Visit https://www.nest-simulator.org

 Type 'nest.help()' to find out more about NEST.



In [2]:
nest.rng_seed = 1
np.random.seed(1)
nest.set_verbosity('M_FATAL')

In [3]:
calcium_data = np.load('/hpc/mzhu843/modelling/nest/DATA/processed/calcium_traces/C_3_p.npy')

spike_data = np.load('/hpc/mzhu843/modelling/nest/DATA/processed/spikes/S_3_p.npy')

ezTrack_data = pd.read_csv('/hpc/mzhu843/modelling/nest/DATA/raw/ezytrack/behaviour_LocationOutput.csv')


categorizer = neuron_categorization.NeuronCategorizer(spike_data, calcium_data, ezTrack_data)
categorizer.run_categorization()
categorized_neurons = categorizer.get_categorized_neurons()
place_obs, int_obs = categorizer.place_obs, categorizer.int_obs

initial_connectivity_matrix = optimization.initialize_connectivity_matrix_normal_distribution(categorized_neurons)

lamb = 0

In [4]:
problem = optimization.SimulatedAnnealing(initial_connectivity_matrix, place_obs, int_obs, lamb, categorized_neurons)
problem.steps = 5000
x, func = problem.anneal()
np.save('/hpc/mzhu843/modelling/nest/DATA/processed/optimised_matrix/optimised_weights', x)

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     2.50000   56527073.09    16.00%    16.00%     1:21:47     0:00:00